In [1]:
import pandas as pd
from pathlib import Path

In [3]:
version = 'no-distil-fine-tune-raw'


In [4]:
basepath = (Path() / 'logs' ).resolve()

In [5]:
df = pd.read_csv(basepath / f'{version}.csv')
ex_df = pd.read_csv(basepath / 'extracted_true_answers.csv')

In [7]:
df['extracted_answer'] = ex_df['extracted_answer']

In [9]:
for i, row in df.iterrows():
    df.loc[i, 'model_answer'] = df['model_answer'][i].removeprefix(row['question'])
    df.loc[i, 'model_answer'] = df['model_answer'][i].removeprefix('\n Please respond with just the answer. The answer is:')
    df.loc[i, 'model_answer'] = df['model_answer'][i].strip()


In [11]:
import re
df['model_ex1'] = df['model_answer']
for i, row in df.iterrows():
    sepstr_extr = re.findall(r'oxed{ *([,\d]+\.?[,\d]*) *}', row['model_answer'])
    first_line = row['model_answer'].split('\n')[0]
    mtches = re.findall(r'[-+]?[$]?([,\d]+\.?[,\d]*)\%?(\ |$|\n|.){1}', first_line)
    osepstr = re.findall(r'(the ?(final|correct)* ?answer ?is)', row['model_answer'])
    oind = row['model_answer'].find(osepstr[0][0]) if osepstr else -1
    oextr = re.findall(r'[-+]?[$]?([,\d]+\.?[,\d]*)\%?(\ |$|\n|.){1}', row['model_answer'][oind:]) if oind != -1 else []
    if sepstr_extr:
        v = re.sub(r',', '', sepstr_extr[0])
    
    elif len(mtches) == 1 or len(set([v[0] for v in mtches])) == 1:
        v = re.sub(r',', '', mtches[0][0])
    elif oextr:
        v = re.sub(r',', '', extr[0][0])
    else:
        extr = re.findall(r'[-+]?[$]?([,\d]+\.?[,\d]*)\%?(\ |$|\n|.){1}', row['model_answer'])
        if len(extr) == 0:
            df.loc[i, 'model_ex1_t'] = type(df['model_ex1'][i])
            continue
        v = re.sub(r',', '', extr[-1][0])
    try:
        df.loc[i, 'model_ex1'] = float(v)
    except:
        df.loc[i, 'model_ex1'] = df['model_ex1'][i]
    df.loc[i, 'model_ex1_t'] = type(df['model_ex1'][i])

In [12]:
df['correct'] = df['model_ex1'] == df['extracted_answer']

In [16]:
df['model_ex1_t'].value_counts()

model_ex1_t
<class 'float'>    1185
<class 'str'>       134
Name: count, dtype: int64

In [18]:
for i, row in df[df['model_ex1_t'] != float].iterrows():
    inp = input(f'{i} : What is the extracted answer : {row["model_ex1"]}. \n \n Question: {row['question']}')
    
    try:
        df.loc[i, 'model_ex1'] = float(inp)
    except:
        if inp == 'no':
            break
        print(f'{i} could not be parsed')

In [19]:
for i, row in df.iterrows():
    df.loc[i, 'model_ex1_t'] = type(row['model_ex1'])


In [21]:
df['correct'] = df['model_ex1'] == df['extracted_answer']

In [ ]:
df['correct'].value_counts()

In [ ]:
df.to_csv(basepath / f'{version.removesuffix('-raw')}.csv')